#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Monthly Budget Mover Parameters
Apply the previous month's budget/spend delta to the current month.  Aggregate up the budget and spend from the previous month of each category declared then apply the delta of the spend and budget equally to each Line Item under that Category.
 1. No changes made can be made in DV360 from the start to the end of this process
 1. Make sure there is budget information for the current and previous month's IOs in DV360
 1. Make sure the provided spend report has spend data for every IO in the previous month
 1. Spend report must contain 'Revenue (Adv Currency)' and 'Insertion Order ID'
 1. There are no duplicate IO Ids in the categories outlined below
 1. This process must be ran during the month of the budget it is updating
 1. If you receive a 502 error then you must separate your jobs into two, because there is too much information being pulled in the sdf
 1. Manually run this job
 1. Once the job has completed go to the table for the new sdf and export to a csv
 1. Take the new sdf and upload it into DV360
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_name': '',  # 
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_read': 'user',  # Credentials used for reading data.
  'partner_id': '',  # The sdf file types.
  'auth_write': 'service',  # Credentials used for writing data.
  'budget_categories': '{}',  # A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}
  'excluded_ios': '',  # A comma separated list of Inserion Order Ids that should be exluded from the budget calculations
  'filter_ids': [],  # Comma separated list of filter ids for the request.
  'version': '5',  # The sdf version to be returned.
  'is_colab': True,  # Are you running this in Colab? (This will store the files in Colab instead of Bigquery)
  'dataset': '',  # Dataset that you would like your output tables to be produced in.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Monthly Budget Mover
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'dataset': {'field': {'order': 1,'name': 'dataset','description': 'Place where tables will be created in BigQuery.','kind': 'string'}},
      'auth': 'user',
      'description': 'Create a dataset where data will be combined and transfored for upload.'
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'order': 7,'name': 'filter_ids','default': '','kind': 'integer_list','description': 'The comma separated list of Advertiser Ids.'}}
          }
        },
        'timeout': 90,
        'body': {
          'timezoneCode': {'field': {'default': 'America/Los_Angeles','name': 'recipe_timezone','description': 'Timezone for report dates.','kind': 'timezone'}},
          'metadata': {
            'format': 'CSV',
            'title': {'field': {'prefix': 'Monthly_Budget_Mover_','name': 'recipe_name','order': 1,'kind': 'string','description': 'Name of report in DV360, should be unique.'}},
            'dataRange': 'PREVIOUS_MONTH'
          },
          'params': {
            'metrics': [
              'METRIC_REVENUE_ADVERTISER'
            ],
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_INSERTION_ORDER'
            ]
          }
        }
      },
      'delete': False
    }
  },
  {
    'monthly_budget_mover': {
      'is_colab': {'field': {'default': True,'name': 'is_colab','order': 7,'kind': 'boolean','description': 'Are you running this in Colab? (This will store the files in Colab instead of Bigquery)'}},
      'sdf': {
        'file_types': 'INSERTION_ORDER',
        'auth': 'user',
        'filter_type': 'FILTER_TYPE_ADVERTISER_ID',
        'time_partitioned_table': False,
        'read': {
          'filter_ids': {
            'values': {'field': {'order': 4,'name': 'filter_ids','default': [],'kind': 'integer_list','description': 'Comma separated list of filter ids for the request.'}},
            'single_cell': True
          }
        },
        'table_suffix': '',
        'dataset': {'field': {'order': 6,'name': 'dataset','default': '','kind': 'string','description': 'Dataset to be written to in BigQuery.'}},
        'create_single_day_table': False,
        'version': {'field': {'name': 'version','description': 'The sdf version to be returned.','kind': 'choice','choices': ['SDF_VERSION_5','SDF_VERSION_5_1'],'order': 6,'default': '5'}},
        'partner_id': {'field': {'order': 1,'name': 'partner_id','description': 'The sdf file types.','kind': 'integer'}}
      },
      'auth': 'user',
      'out_old_sdf': {
        'file': '/content/old_sdf.csv',
        'bigquery': {
          'dataset': {'field': {'order': 8,'name': 'dataset','default': '','kind': 'string','description': 'Dataset that you would like your output tables to be produced in.'}},
          'skip_rows': 0,
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'table': {'field': {'prefix': 'SDF_OLD_','name': 'recipe_name','description': '','kind': 'string'}}
        }
      },
      'excluded_ios': {'field': {'order': 4,'name': 'excluded_ios','description': 'A comma separated list of Inserion Order Ids that should be exluded from the budget calculations','kind': 'integer_list'}},
      'report_name': {'field': {'prefix': 'Monthly_Budget_Mover_','name': 'recipe_name','order': 1,'kind': 'string','description': 'Name of report in DV360, should be unique.'}},
      'out_changes': {
        'file': '/content/log.csv',
        'bigquery': {
          'dataset': {'field': {'order': 8,'name': 'dataset','default': '','kind': 'string','description': 'Dataset that you would like your output tables to be produced in.'}},
          'skip_rows': 0,
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'table': {'field': {'prefix': 'SDF_BUDGET_MOVER_LOG_','name': 'recipe_name','description': '','kind': 'string'}}
        }
      },
      'budget_categories': {'field': {'order': 3,'name': 'budget_categories','default': '{}','kind': 'json','description': 'A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}'}},
      'out_new_sdf': {
        'file': '/content/new_sdf.csv',
        'bigquery': {
          'dataset': {'field': {'order': 8,'name': 'dataset','default': '','kind': 'string','description': 'Dataset that you would like your output tables to be produced in.'}},
          'skip_rows': 0,
          'schema': [
          ],
          'disposition': 'WRITE_TRUNCATE',
          'table': {'field': {'prefix': 'SDF_NEW_','name': 'recipe_name','description': '','kind': 'string'}}
        }
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
